# Book 2 - Selecionamento de Features, Validação, Balanceamento

Este notebook serviu como registro prático e teórico no meu aprendizado de Machine Learning.

`Enriqueci este notebook com anotações adicionais e aplicações práticas tornando-o uma referência valiosa para consultas e implementações em futuros projetos reais.`

Espero que este material inspire outros a explorar ainda mais o fascinante mundo do Machine Learning. 

No notebook presente tem todos os topicos dos notebook anteriores, porém sendo acrescentado e aprofundado com anotações dos seguintes tópicos:

**Técnicas de Balanceamento de Dados**  
- **Oversampling - Upsampling**: Criação de dados sintéticos.
- **Undersampling - Downsampling**: Redução de amostras na classe majoritária.

Compartilhar conhecimento é uma alegria—viva ao aprendizado contínuo, boa pratica e bons estudo a quem estiver lendo, abraços!

# Funções, bibliotecas e Dataframe ficticios

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

# Manipulação e Tratamento de dados
import openpyxl
import pandas as pd
import numpy as np
from numpy import NaN

#ignorando Warning inuteis
import warnings 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
# Carregar os dados
url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(url)

# Identificar colunas a serem removidas # Remover colunas inúteis
columns_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin']
df = df.drop(columns=columns_to_drop)
df = df.dropna()

df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB


In [4]:
colunas_cat = ['Pclass','Sex','Embarked']
for coluna in colunas_cat:
    df[coluna] = df[coluna].astype('O')

In [5]:
x = df.drop('Survived', axis=1)
y = df['Survived']

In [6]:
# DUMMYRIZAÇÃO
colunas_categoricas = []
colunas_binarias = []
colunas_mais3_categorias = []

for coluna in x.columns:
    if df[coluna].dtype == 'O':
        categorias = x[coluna].unique()
        if len(categorias) == 2:
            print('2 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_binarias.append(coluna)
        else:
            print('3 niveis:', coluna, '=>', categorias)
            colunas_categoricas.append(coluna)
            colunas_mais3_categorias.append(coluna)

############################################################################################
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder #transformando colunas com 2 categorias em 0 e 1

coluna = x.columns
one_hot = make_column_transformer((
    OneHotEncoder(drop='if_binary'), #caso a coluna tenha apenas 2 categorias 
    colunas_categoricas), #passando quais são essas colunas
    remainder = 'passthrough', sparse_threshold=0) #oque deve ser feito com as outras

#Aplicando transformação
x = one_hot.fit_transform(x)

#Os novos nomes das colunas #'onehotencoder=transformadas; 'remainder'=não transformadas
novos_nomes_colunas = one_hot.get_feature_names_out(coluna)

# Remover prefixo 'remainder__' das colunas que não foram transformadas
#novos_nomes_colunas = [nome.replace('remainder__', '') for nome in novos_nomes_colunas]

x = pd.DataFrame(x, columns = novos_nomes_colunas) #alterando de volta
x_columns = x.columns.tolist() 

############################################################################################
# Normalização (scaling entre 0 e 1) com MinMaxScaler ******************************
from sklearn.preprocessing import MinMaxScaler
normalizacao = MinMaxScaler()
#x = normalizacao.fit_transform(x)
# df['Close_normalizada'] = (df[coluna] - df[coluna].min()) / (df[coluna].max() - df[coluna].min())

# Padronização (média 0 e desvio padrão 1) com StandardScaler **********************
from sklearn.preprocessing import StandardScaler
padronizacao = StandardScaler()
#x = padronizacao.fit_transform(x)
# df['Close_padronizada'] = (df[coluna] - df[coluna].mean()) / df[coluna].std()

############################################################################################
# DEFININDO A VARIAVEL DEPENDENTE
from sklearn.preprocessing import LabelEncoder
y = LabelEncoder().fit_transform(y)

############################################################################################
#backups
x_inteiro = x
y_inteiro = y

# DIVIDINDO BASE EM TREINO E TESTE
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, 
                                                    stratify = y, #para manter a proporção da Var Dep nos splits
                                                    random_state = 5) #raiz da aleatoridade
# test_size = 0.25 #porcentagem que ira ser separado para testes

print(x_treino.shape, x_teste.shape)
print(y_treino.shape, y_teste.shape)

3 niveis: Pclass => [3 1 2]
2 niveis: Sex => ['male' 'female']
3 niveis: Embarked => ['S' 'C' 'Q']
(534, 11) (178, 11)
(534,) (178,)


# ====================================

# Balanceamento de Dados

# ====================================

# Oversampling - Upsampling

Abordagem avançada de oversampling usada para balancear conjuntos de dados desbalanceados, aumentando a representatividade das classes minoritárias por meio da `criação de dados sintéticos`, ao invés de simplesmente replicar os exemplos existentes.

(Exemplo SMOTE)

- **Diversificação:** Ao gerar novos exemplos, SMOTE introduz uma variedade maior no conjunto de dados, o que pode ajudar a evitar o overfitting que poderia ocorrer se simplesmente duplicássemos as amostras existentes.
- **Melhoria de Modelagem:** Com um balanceamento mais efetivo entre as classes, os modelos são capazes de aprender padrões mais generalizáveis, melhorando assim a precisão das previsões em dados não vistos.

SMOTE é amplamente utilizado em problemas de classificação onde o desequilíbrio de classes é significativo, como em detecção de fraude, diagnóstico médico e predição de falhas em equipamentos.

**Funcionamento**

1. **Identificação das Amostras:**
   SMOTE analisa as características das amostras minoritárias (classe sub-representada) e identifica seus vizinhos mais próximos.

2. **Síntese de Novos Exemplos:**
   Para cada amostra na classe minoritária, são criados novos exemplos sintéticos. Isso é feito selecionando um dos \( k \) vizinhos mais próximos (geralmente \( k=5 \)) e interpolando um novo ponto entre a amostra original e o vizinho selecionado.

3. **Adição ao Conjunto de Dados:**
   Os exemplos sintéticos gerados são então adicionados ao conjunto de dados, aumentando a proporção da classe minoritária.

**Considerações**

- **Espaço de Características:** SMOTE funciona bem quando as características são contínuas. Em dados categóricos, outras técnicas de oversampling, como o ADASYN (Adaptive Synthetic Sampling Approach), podem ser mais apropriadas.
- **Risco de Overfitting:** Apesar de introduzir diversidade, a criação de muitos exemplos sintéticos pode levar a um modelo excessivamente otimista em relação aos dados de treinamento. Deve-se ter cautela com o número de exemplos sintéticos gerados.
- **Não Adiciona Novas Informações:** Como as amostras são apenas replicadas, nenhuma informação nova é introduzida ao modelo, o que pode limitar a capacidade do modelo de aprender nuances mais complexas das classes.
- **Combinação com Downsampling:** Frequentemente, o upsampling é combinado com o downsampling da classe majoritária para criar um equilíbrio ainda mais efetivo e evitar o aumento excessivo do conjunto de dados.


In [ ]:
from sklearn.utils import resample

# Separando por classe
x_inteiro_df = pd.DataFrame(x_inteiro)
y_inteiro_series = pd.Series(y_inteiro)
classe_maior = x_inteiro_df.iloc[sorted(y_inteiro_series[y_inteiro_series == 1].index)]
classe_menor = x_inteiro_df.iloc[sorted(y_inteiro_series[y_inteiro_series == 0].index)]

# Upsampling da classe minoritária
train_df_menor_upsampled = resample(classe_menor,
                                    replace=True,                # sample with replacement
                                    n_samples=len(classe_menor), # to match majority class
                                    random_state=123)            # reproducible results

# Combinando a classe majoritária com a classe minoritária upsampled
train_df_upsampled = pd.concat([classe_menor, train_df_menor_upsampled])

## SMOTE (Synthetic Minority Over-sampling Technique)
Gera novos exemplos sintéticos da classe minoritária ao invés de duplicar exemplos existentes. Funciona interpolando entre os exemplos minoritários e criando novos pontos ao longo das linhas que ligam os vizinhos mais próximos.

In [ ]:
# SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_treino_balanceado, y_treino_balanceado = smote.fit_resample(x_treino, y_treino)


## Variantes do SMOTE:

### Borderline-SMOTE
Aplica oversampling apenas nos exemplos da classe minoritária que estão perto da fronteira com a classe majoritária.

In [ ]:
# Borderline-SMOTE
from imblearn.over_sampling import BorderlineSMOTE
borderline_smote = BorderlineSMOTE()
x_treino_balanceado, y_treino_balanceado = borderline_smote.fit_resample(x_treino, y_treino)


### SMOTE-ENN (Edited Nearest Neighbours)
Combina SMOTE com um método de limpeza dos dados chamado ENN, que remove exemplos ruidosos após o oversampling.

In [ ]:
# SMOTE-ENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN()
x_treino_balanceado, y_treino_balanceado = smote_enn.fit_resample(x_treino, y_treino)


### ADASYN (Adaptive Synthetic Sampling)
Variante do SMOTE que ajusta o número de exemplos sintéticos gerados com base na densidade local, gerando mais exemplos para minorias que estão cercadas por muitas instâncias da classe majoritária.

In [ ]:
# ADASYN
from imblearn.over_sampling import ADASYN
adasyn = ADASYN()
x_treino_balanceado, y_treino_balanceado = adasyn.fit_resample(x_treino, y_treino)


### K-means SMOTE
Uma variante do SMOTE que usa agrupamento (k-means clustering) antes de aplicar o SMOTE, gerando exemplos sintéticos baseados nos clusters de dados minoritários.

In [ ]:
# K-means SMOTE
from imblearn.over_sampling import KMeansSMOTE
kmeans_smote = KMeansSMOTE()
x_treino_balanceado, y_treino_balanceado = kmeans_smote.fit_resample(x_treino, y_treino)


### Random Oversampling
Duplica aleatoriamente exemplos da classe minoritária até atingir um balanço desejado. Isso pode levar à overfitting, já que os mesmos exemplos são repetidos.

In [ ]:
from imblearn.over_sampling import RandomOverSampler
oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(x_treino, y_treino)

# Undersampling - Downsampling

Em resumo: Reduzem o número de exemplos da classe majoritária, removendo instâncias de forma controlada para equilibrar o conjunto.

Este método específico foca na `redução da classe majoritária`, mas com uma abordagem mais refinada que simplesmente remover amostras aleatoriamente. O NearMiss seleciona amostras da classe majoritária baseado em certos critérios de proximidade, com o objetivo de manter apenas aquelas que são mais representativas e/ou mais próximas das amostras da classe minoritária.

**Funcionamento (NearMiss)**

1. **Critérios de Seleção:**
   NearMiss implementa diferentes versões de seleção:
   - **NearMiss-1:** Seleciona amostras da classe majoritária com a menor distância média às três amostras mais próximas da classe minoritária.
   - **NearMiss-2:** Seleciona amostras da classe majoritária com a menor distância média às três amostras mais distantes da classe minoritária.
   - **NearMiss-3:** Um subconjunto da classe minoritária é selecionado primeiro, e então, para cada exemplo na classe minoritária, são retidas as \( n \) amostras mais próximas da classe majoritária.

2. **Redução da Classe Majoritária:**
   Amostras são selecionadas de acordo com o critério estabelecido até que o número de instâncias na classe majoritária seja reduzido suficientemente para igualar o da classe minoritária.

3. **Combinação de Dados:**
   As amostras da classe majoritária que atendem aos critérios são combinadas com as da classe minoritária para formar um novo conjunto de dados balanceado.

**Considerações**

- **Perda de Informação Crítica:** Apesar da intenção de manter amostras importantes, a remoção de grandes quantidades de dados pode resultar em perda de informações cruciais.
- **Escolha do Método:** A escolha entre NearMiss-1, NearMiss-2, e NearMiss-3 pode ter um impacto significativo nos resultados, exigindo testes para determinar qual método se adapta melhor ao problema específico.
- **Escolha de Amostras:** A seleção aleatória de amostras para remoção pode não ser a abordagem ideal; métodos mais sofisticados podem ser necessários para preservar a integridade da informação.
- **Combinação com Upsampling:** Muitas vezes, o downsampling é usado em conjunto com o upsampling para não apenas reduzir a classe majoritária, mas também aumentar a minoritária, alcançando um equilíbrio ideal.
- **Técnicas Avançadas:** Métodos como clustering ou análises de importância de instâncias podem ser utilizados para escolher quais amostras remover, assegurando que as mais representativas e informativas sejam mantidas.

## Random Undersampling
Remove exemplos da classe majoritária aleatoriamente. Embora simples, pode resultar na perda de informações importantes se não for usado com cuidado.

In [ ]:
# Random Undersampling
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
x_treino_balanceado, y_treino_balanceado = rus.fit_resample(x_treino, y_treino)


## NearMiss
Seleciona exemplos da classe majoritária que estão mais próximos dos exemplos da classe minoritária, tentando manter exemplos representativos da classe majoritária.

> NearMiss-1: 
    Escolhe exemplos da classe majoritária com a menor distância média para os três exemplos mais próximos da classe minoritária.

    
> NearMiss-2:
    Escolhe exemplos da classe majoritária com a menor distância média para todos os exemplos da classe minoritária.

In [ ]:
# NearMiss
from imblearn.under_sampling import NearMiss
near_miss = NearMiss(version=3)
x_treino_balanceado, y_treino_balanceado = near_miss.fit_resample(x_treino, y_treino)

## Tomek Links
Identifica pares de exemplos (um da classe majoritária e um da minoritária) que são vizinhos mais próximos e pertencem a classes diferentes. Se esses pares forem encontrados, o exemplo da classe majoritária é removido, limpando a fronteira entre as classes.

In [ ]:
# Tomek Links
from imblearn.under_sampling import TomekLinks
tomek_links = TomekLinks()
x_treino_balanceado, y_treino_balanceado = tomek_links.fit_resample(x_treino, y_treino)

## Cluster Centroids
Uma técnica de undersampling baseada em clusterização, onde os dados da classe majoritária são agrupados e os centróides desses clusters substituem os exemplos originais. Isso reduz o número de exemplos da classe majoritária sem perder muita representatividade

In [ ]:
# Cluster Centroids
from imblearn.under_sampling import ClusterCentroids
cluster_centroids = ClusterCentroids()
x_treino_balanceado, y_treino_balanceado = cluster_centroids.fit_resample(x_treino, y_treino)

# Técnicas Combinadas (Over/Under Sampling)

## SMOTE + Tomek Links
Primeiro aplica SMOTE para gerar exemplos sintéticos da classe minoritária e depois aplica Tomek Links para remover exemplos da classe majoritária que estão muito próximos da classe minoritária.

In [ ]:
# SMOTE + Tomek Links
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek()
x_treino_balanceado, y_treino_balanceado = smote_tomek.fit_resample(x_treino, y_treino)


## SMOTE + NearMiss
Combina SMOTE para a classe minoritária com NearMiss para a classe majoritária, criando um equilíbrio mais controlado entre as classes.

In [ ]:
# SMOTE + NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

# Aplicando SMOTE para oversampling
smote = SMOTE()
x_treino_oversampled, y_treino_oversampled = smote.fit_resample(x_treino, y_treino)

# Aplicando NearMiss para undersampling
near_miss = NearMiss(version=3)
x_treino_balanceado, y_treino_balanceado = near_miss.fit_resample(x_treino_oversampled, y_treino_oversampled)


# Técnicas Baseadas em Algoritmos
Alguns algoritmos de machine learning possuem abordagens internas para lidar com dados desbalanceados.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Aplicando pesos de classe para balanceamento automático
clf = LogisticRegression(class_weight='balanced')
clf.fit(x_treino, y_treino)


## BalancedRandomForest
Uma técnica baseada em árvores de decisão que cria várias árvores com conjuntos de dados balanceados. Em cada árvore, realiza undersampling da classe majoritária de forma aleatória.

In [ ]:
# Balanced Random Forest
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier()
brf.fit(x_treino, y_treino)


In [ ]:
# Balanced Bagging Classifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10)
bbc.fit(x_treino, y_treino)


# Técnicas de Amostragem Informatizada

## Edited Nearest Neighbors (ENN)
Remove exemplos da classe majoritária e da minoritária que são mal classificados pelos seus vizinhos mais próximos, ajudando a melhorar a separação entre as classes.

In [ ]:
# Edited Nearest Neighbors (ENN)
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours()
x_treino_balanceado, y_treino_balanceado = enn.fit_resample(x_treino, y_treino)


## Repeated Edited Nearest Neighbors (RENN)
Aplica o processo de ENN repetidamente até que nenhum exemplo seja removido, limpando ainda mais as fronteiras entre as classes.

In [ ]:
# Repeated Edited Nearest Neighbours (RENN)
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
renn = RepeatedEditedNearestNeighbours()
x_treino_balanceado, y_treino_balanceado = renn.fit_resample(x_treino, y_treino)


## One-Sided Selection (OSS)
Combina Tomek Links com ENN para limpar a classe majoritária, removendo exemplos ruidosos e fronteiriços.

In [ ]:
# One-Sided Selection (OSS)
from imblearn.under_sampling import OneSidedSelection
oss = OneSidedSelection()
x_treino_balanceado, y_treino_balanceado = oss.fit_resample(x_treino, y_treino)


# FIM